In [66]:
import pandas as pd
import numpy as np
import torch

Save to Pandas DF

In [67]:
raw_data = pd.read_csv('data/energy_dataset.csv')
raw_data['time'] = pd.to_datetime(raw_data['time'], utc=True)
raw_data.set_index('time', inplace=True)

In [68]:
raw_data.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


Fill in missing values

In [69]:
def strategic_fill(df):
    """
    Fills missing values in the DataFrame based on the following policy:
    - For each missing hour, check the same hour from the previous day.
      If it exists, use it to fill the missing value.
    - If the previous day's same hour is also missing, check the next day's same hour.
      If it exists, use it to fill the missing value.
    - If both previous and next day are missing, look further back and forward
      up to 7 days in both directions.
    - Finally, use forward and backward fill for any remaining missing values.
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df_filled = df.copy()
    
    # First pass: try to fill with same hour from adjacent days
    for time in df_filled.index[df_filled.isnull().any(axis=1)]:
        # Get columns that have missing values for this timestamp
        missing_cols = df_filled.columns[df_filled.loc[time].isnull()]
        
        # Try previous day first
        previous_day = time - pd.Timedelta(days=1)
        if previous_day in df_filled.index:
            # Fill only the missing columns from previous day
            for col in missing_cols:
                if not pd.isnull(df_filled.loc[previous_day, col]):
                    df_filled.loc[time, col] = df_filled.loc[previous_day, col]
        
        # Check which columns are still missing
        still_missing = df_filled.columns[df_filled.loc[time].isnull()]
        
        # If some columns are still missing, try next day
        if len(still_missing) > 0:
            next_day = time + pd.Timedelta(days=1)
            if next_day in df_filled.index:
                # Fill only the still missing columns from next day
                for col in still_missing:
                    if not pd.isnull(df_filled.loc[next_day, col]):
                        df_filled.loc[time, col] = df_filled.loc[next_day, col]
    
    # Second pass: look further back and forward up to 7 days
    for time in df_filled.index[df_filled.isnull().any(axis=1)]:
        missing_cols = df_filled.columns[df_filled.loc[time].isnull()]
        
        for days in range(2, 8):  # Try 2 to 7 days back
            if len(missing_cols) == 0:
                break
                
            prev_day = time - pd.Timedelta(days=days)
            if prev_day in df_filled.index:
                for col in missing_cols:
                    if not pd.isnull(df_filled.loc[prev_day, col]):
                        df_filled.loc[time, col] = df_filled.loc[prev_day, col]
                        
            # Update missing columns list
            missing_cols = df_filled.columns[df_filled.loc[time].isnull()]
            
            if len(missing_cols) == 0:
                break
                
            next_day = time + pd.Timedelta(days=days)
            if next_day in df_filled.index:
                for col in missing_cols:
                    if not pd.isnull(df_filled.loc[next_day, col]):
                        df_filled.loc[time, col] = df_filled.loc[next_day, col]
                        
            # Update missing columns list again
            missing_cols = df_filled.columns[df_filled.loc[time].isnull()]
    
    # Final pass: use forward fill followed by backward fill for any remaining missing values
    df_filled = df_filled.ffill().bfill()
    
    return df_filled


In [70]:
start_time = raw_data.index.min()
end_time = raw_data.index.max()
complete_range = pd.date_range(start=start_time, end=end_time, freq='h')
# Find missing hours and then insert these times to make the time consecutive
missing_hours = complete_range.difference(raw_data.index)
print(missing_hours)
missing_df = pd.DataFrame(index=missing_hours)
raw_data = pd.concat([raw_data, missing_df])
raw_data = raw_data.sort_index()

filled_df = strategic_fill(raw_data)

DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='h')


In [71]:
filled_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,NaN,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,NaN,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,NaN,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,NaN,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,NaN,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [72]:
columns_to_drop = ['generation hydro pumped storage aggregated', 'forecast wind onshore day ahead', 'forecast wind offshore eday ahead']
filled_df = filled_df.drop(columns=columns_to_drop, errors='ignore')

In [73]:
filled_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,total load forecast,total load actual,price day ahead,price actual
time,,,,,,,,,,,,,,,,,,,,,
2014-12-31 23:00:00+00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,863.0,...,73.0,49.0,196.0,0.0,6378.0,17.0,26118.0,25385.0,50.10,65.41
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,920.0,...,71.0,50.0,195.0,0.0,5890.0,16.0,24934.0,24382.0,48.10,64.92
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,1164.0,...,73.0,50.0,196.0,0.0,5461.0,8.0,23515.0,22734.0,47.33,64.48
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,1503.0,...,75.0,50.0,191.0,0.0,5238.0,2.0,22642.0,21286.0,42.27,59.32
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,1826.0,...,74.0,42.0,189.0,0.0,4935.0,9.0,21785.0,20264.0,38.41,56.04


In [74]:
# Get indices where there are still null values
null_indices = filled_df.index[filled_df.isnull().any(axis=1)]
print("Indices with null values:")
print(null_indices)

# Print the columns for each of these indices
print("\nColumns at each index with null values:")
for idx in null_indices:
    print(f"\nIndex: {idx}")
    print(filled_df.loc[idx])

Indices with null values:
DatetimeIndex([], dtype='datetime64[ns, UTC]', name='time', freq=None)

Columns at each index with null values:


Add dummy variables

In [75]:
# Now extract time components
filled_df['HoD'] = filled_df.index.hour  # Hour of the Day (0 to 23)
# Day of the Week (1=Monday to 7=Sunday)
filled_df['DoW'] = filled_df.index.dayofweek + 1
filled_df['MoY'] = filled_df.index.month

In [76]:
filled_df = filled_df.iloc[1:]
filled_df[['HoD', 'DoW', 'MoY']].head()

,HoD,DoW,MoY
time,,,
2015-01-01 00:00:00+00:00,0,4,1
2015-01-01 01:00:00+00:00,1,4,1
2015-01-01 02:00:00+00:00,2,4,1
2015-01-01 03:00:00+00:00,3,4,1
2015-01-01 04:00:00+00:00,4,4,1


In [77]:
len(filled_df)

35063

In [78]:
filled_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation wind offshore,generation wind onshore,forecast solar day ahead,total load forecast,total load actual,price day ahead,price actual,HoD,DoW,MoY
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,920.0,...,0.0,5890.0,16.0,24934.0,24382.0,48.10,64.92,0,4,1
2015-01-01 01:00:00+00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,1164.0,...,0.0,5461.0,8.0,23515.0,22734.0,47.33,64.48,1,4,1
2015-01-01 02:00:00+00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,1503.0,...,0.0,5238.0,2.0,22642.0,21286.0,42.27,59.32,2,4,1
2015-01-01 03:00:00+00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,1826.0,...,0.0,4935.0,9.0,21785.0,20264.0,38.41,56.04,3,4,1
2015-01-01 04:00:00+00:00,410.0,178.0,0.0,4038.0,3590.0,156.0,0.0,0.0,0.0,2109.0,...,0.0,4618.0,4.0,21441.0,19905.0,35.72,53.63,4,4,1


Shift forecast cols

In [86]:
shifted_df = filled_df.copy()
frc_cols = [col for col in shifted_df.columns if "forecast" in col]
print(frc_cols)
for col in frc_cols:
    if col in shifted_df.columns:
        shifted_df[col] = shifted_df[col].shift(-24)
shifted_df.dropna(subset=frc_cols, inplace=True)

['forecast solar day ahead', 'total load forecast']


In [87]:
shifted_df.head()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation wind offshore,generation wind onshore,forecast solar day ahead,total load forecast,total load actual,price day ahead,price actual,HoD,DoW,MoY
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+00:00,368.0,0.0,0.0,2902.0,1190.0,192.0,0.0,0.0,0.0,1996.0,...,0.0,12760.0,16.0,24934.0,24935.0,7.0,58.46,0,5,1
2015-01-02 01:00:00+00:00,358.0,0.0,0.0,2772.0,1023.0,189.0,0.0,0.0,0.0,2698.0,...,0.0,12174.0,8.0,23515.0,23214.0,5.0,54.70,1,5,1
2015-01-02 02:00:00+00:00,353.0,0.0,0.0,2936.0,1016.0,188.0,0.0,0.0,0.0,3269.0,...,0.0,11358.0,2.0,22642.0,22540.0,4.0,54.91,2,5,1
2015-01-02 03:00:00+00:00,354.0,0.0,0.0,2893.0,1103.0,189.0,0.0,0.0,0.0,3267.0,...,0.0,11199.0,9.0,21785.0,22096.0,4.0,53.07,3,5,1
2015-01-02 04:00:00+00:00,354.0,0.0,0.0,2889.0,1120.0,190.0,0.0,0.0,0.0,3258.0,...,0.0,11152.0,4.0,21441.0,22066.0,4.8,54.23,4,5,1


In [89]:
len(shifted_df)

35039

In [88]:
shifted_df.to_csv('data/cleaned_spain_data.csv')